In [1]:
from HumanChecker_model import HumanChecker
import torch
from torch.utils import data as DataUtil
import config
import h5py
import pytorch_lightning as pl

In [2]:
tr_p,val_p = 0.9,0.1
class original_data(DataUtil.Dataset):
    def __init__(self) -> None:
        super().__init__()

        with h5py.File('data/HumanChecker.h5','r') as f:
            voices = torch.from_numpy(f['voice'][...])
            noizes = torch.from_numpy(f['noize'][...])
        self.data = torch.cat([voices,noizes])
        self.ans = torch.cat([torch.ones(voices.size(0)),torch.zeros(noizes.size(0))]).unsqueeze(-1).half()
        print(self.data.shape,self.ans.shape,self.data.dtype,self.ans.dtype)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index],self.ans[index]

orig = original_data()
trlen = int(len(orig)*tr_p)
vlen = len(orig) - trlen
trainset,valset = DataUtil.random_split(orig,[trlen,vlen])

torch.Size([3299, 321, 32]) torch.Size([3299, 1]) torch.float16 torch.float16


In [3]:
model = HumanChecker()
batch_size = 64
EPOCHS = 50
train_loader = DataUtil.DataLoader(trainset,batch_size,shuffle=True)
val_loader = DataUtil.DataLoader(valset,batch_size,shuffle=False)

In [4]:
trainer = pl.Trainer(gpus=1,num_nodes=1,precision=16,max_epochs=EPOCHS)
trainer.fit(model,train_loader,val_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type              | Params
-------------------------------------------------
0 | criterion  | BCEWithLogitsLoss | 0     
1 | init_dense | Linear            | 103 K 
2 | conformers | ModuleList        | 8.6 M 
3 | dense      | Linear            | 10.2 K
-------------------------------------------------
8.7 M     Trainable params
0         Non-trainable params
8.7 M     Total params
34.698    Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\22shi\anaconda3\envs\JARVIS\lib\site-packages\pytorch_lightning\trainer\data_loading.py:102: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 0/53 [00:00<?, ?it/s] 

C:\Users\22shi\anaconda3\envs\JARVIS\lib\site-packages\pytorch_lightning\trainer\data_loading.py:102: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 6:   9%|▉         | 5/53 [00:00<00:03, 13.33it/s, loss=0.000635, v_num=7]

C:\Users\22shi\anaconda3\envs\JARVIS\lib\site-packages\pytorch_lightning\trainer\trainer.py:897: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn('Detected KeyboardInterrupt, attempting graceful shutdown...')


Epoch 6:   9%|▉         | 5/53 [00:13<02:08,  2.68s/it, loss=0.000635, v_num=7]

In [ ]:
from datetime import datetime
now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S-%f')
name = f'params/humanchecker{now}.params'
torch.save(model.state_dict(),name)
print('saved')